In [ ]:
using JuMP, Gurobi, PyPlot, DataFrames, CSV

# Solving the p-median problem by Lagrangian Relaxation
p = 3

# Reading demand data
d, header = CSV.read("demand.csv",  header=true)

# Reading transportation cost data
data = CSV.read("cost.csv")
c = data[2:end, 2:end]
c = convert(Array{Float64,2}, c)

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1192


In [6]:
# the length of 'd' and the number of columns in 'c' must match
@assert length(d) == size(c,2)

locations = 1:size(c,1) # the set, I
customers = 1:length(d) # the set, J



UndefVarError: UndefVarError: d not defined

In [ ]:

function optimal(p)
  m = Model(solver=GurobiSolver())

  @variable(m, x[i in locations, j in customers] >= 0)
  @variable(m, y[i in locations], Bin)

  @objective(m, Min, sum( d[j]*c[i,j]*x[i,j]
                     for i in locations, j in customers) )

  for j in customers
    @constraint(m, sum( x[i,j] for i in locations) == 1)
  end

  @constraint(m, sum( y[i] for i in locations) == p)

  for i in locations
    for j in customers
      @constraint(m, x[i,j] <= y[i] )
    end
  end

  solve(m)

  Z_opt = getobjectivevalue(m)
  x_opt = getvalue(x)
  y_opt = getvalue(y)

  return Z_opt, x_opt, y_opt
end